In [1]:
import random
from IPython.display import clear_output
import sys

#Constants
BET_RATIO = 2 # Player will win bet money * BET_RATIO
DEALERS_THRESHOLD = 17 # The dealer will hit until he gets to this value or better

class Card():   
    def __init__(self, category, name, value):
        # private attributes, Python uses the __ as convention
        self.__category = category 
        self.__name = name
        self.__value = value
        
    def __str__(self):
        return f"{self.__name} of {self.__category} ({self.__value})"
    
    def set_value(self, value):
        self.__value = value
        
    def get_value(self):
        return self.__value
    
    def get_name(self):
        return self.__name
    
    def get_category(self):
        return self.__category


class Deck():
    CARD_CATEGORIES = ["Spades", "Hearts", "Diamonds", "Clubs"]
    CARD_VALUES = {"Ace":[1,11], "Two":2, "Three":3, "Four":4, "Five":5, "Six":6, "Seven":7, "Eight":8, "Nine":9, "Ten":10, "Jack":10, "Queen":10, "King":10}
    
    def __init__(self):
        self.cards = []
        self.total_cards = 52

        # Create our deck for blackjack and shuffle it
        for card in self.CARD_VALUES:
            for category in self.CARD_CATEGORIES:
                self.cards.append(Card(category, card, self.CARD_VALUES[card]))

        # Shuffle our deck
        self.shuffle()
          
    def shuffle(self):
        random.shuffle(self.cards)
    
    def top_card(self):
        # Takes the first card of the deck and returns it
        # No need to control the case of the deck being empty, this will never happen in this game
        self.total_cards-=1
        return self.cards.pop(0)
    
    def __str__(self):
        # Pretty sure this is not the way to do this LUL
        print(f"Total cards:{self.total_cards}")
        for card in self.cards:
            print(card)
        return ""
    
class Hand():
    def __init__(self):
        self.__cards = [] # private attribute
    
    def draw(self, deck):
        self.__cards.append(deck.top_card())
    
    def total_value(self):
        return sum(card.get_value() for card in self.__cards)
    
    def get_cards(self):
        return self.__cards


# TODO: this should be an abstract class
class Player():
    def __init__(self, savings):
        self.savings = savings
        self._hand = Hand()
        pass
      
    def draw(self,deck):
        self._hand.draw(deck)
    
    def print_hand(self, hide_second_card=False):
        # Prints the hand of the player
        if hide_second_card:
            for index,card in enumerate(self._hand.get_cards()):
                if index==1:
                    print("X|", end = '')
                else:
                    print(f"{card}|", end = '')
        else:
            for card in self._hand.get_cards():
                print(f"{card}|", end = '') 
                
    def busted(self):
        return self._hand.total_value() > 21
    
    def hand_value(self):
        return self._hand.total_value()
    
    def define_aces_values(self):
        pass
    
    def win(self, amount=0):
        pass
    
    def lose(self, amount=0):
        pass
    
    def reset_hand(self):
        self._hand = Hand()
    

class HumanPlayer(Player):
    def __init__(self,savings=100):
        Player.__init__(self,savings)
        self.bet_money = 0
    
    def can_bet(self,amount):
        return amount <= self.savings
    
    def bet(self,amount):
        # Update bet money here
        self.savings -= amount
        self.bet_money = amount
        
    def define_aces_values(self):
        # Implement this method, the player has to decide if he will use their aces
        # as 1 or 11
        for card in self._hand.get_cards():
            condition = card.get_name() == "Ace" and card.get_value() == [1,11]            
            if(condition):
                # We need to ask the player to define the value for this card and update it
                ace_value = Utils.get_ace_value(card)
                card.set_value(ace_value)
                
    def reset_bet(self):
        self.savings += self.bet_money
        self.bet_money = 0
        
    def win(self,amount=0):
        self.savings += self.bet_money*BET_RATIO
        self.bet_money = 0
        
    def lose(self, amount=0):
        # Savings is modified right when we start and he bets
        self.bet_money = 0
        
    
class Dealer(Player):
    def __init__(self, savings=sys.maxsize):
        Player.__init__(self,savings)
        
    def define_aces_values(self):
        # I'm gonna implement this method randomly, with the only consideration that
        # If there's already an ace in the hand with the value of 11, the rest will be
        # considered as 1        
        for card in self._hand.get_cards():
            ace_not_set = card.get_name() == "Ace" and card.get_value() == [1,11]            
            if(ace_not_set):
                if(any(card.get_value() == 11 for card in self._hand.get_cards())):
                    # Consider this ace as one
                    card.set_value(1)
                else:
                    # Generate a random value for it
                    random_value = random.randint(0,1)
                    if random_value:
                        card.set_value(1)
                    else:
                        card.set_value(11)
    
    def win(self,amount=0):
        self.savings += amount
        
    def lose(self, amount=0):
        self.savings -= amount
    
class Utils:
    def capture_bet():
        while True:
            try:
                bet = int(input("Place your bet: "))
                return bet
            except:
                continue
    
    '''
    Return 0 if the player decides to stand or 1 if he wants to hit 
    '''
    def wants_to_hit():
        while True:
            hit_or_stand = input("Would you like to hit[h] or stand[s]?")
            if(hit_or_stand.lower() in ["hit", "h"]):
                return 1
            elif(hit_or_stand.lower() in ["stand", "s"]):
                return 0
            else:
                print("Invalid input, only 'h'/'hit' or 's'/'stand' are valid")
                continue

    def print_state(deck, dealer, player1, hide_second_card=False):
        clear_output()
        # Prints out the board with the state
        print("")
        print("Dealer")
        print("--------")
        print(f"Cards in deck: {deck.total_cards}")
        print(f"Savings: ${dealer.savings}")
        dealer.print_hand(hide_second_card)

        print("")
        print("")

        print("Player1")
        print("--------")
        print(f"Savings: ${player1.savings}")  
        print(f"Bet: ${player1.bet_money}")
        player1.print_hand()
        print("")
    
    def get_ace_value(card):
        while True:
            try:
                ace_value = int(input(f"Will you use {card.get_name()} of {card.get_category()} as 1 or 11?:"))

                if ace_value not in [1,11]:
                    print("Invalid value. It should be 1 or 11!")
                    pass
                    continue
                else:
                    return ace_value
            except:
                continue
                
    def player1_wins(deck, dealer, player1):
        dealer.lose(player1.bet_money*BET_RATIO)
        player1.win()
        Utils.print_state(deck, dealer, player1, False)
        print("")
        print("PLAYER1 YOU'VE WON THE GAME!")
    
    def dealer_wins(deck, dealer, player1):
        dealer.win(player1.bet_money)
        player1.lose()
        Utils.print_state(deck, dealer, player1, False)
        print("")
        print("PLAYER1 YOU'VE LOST THE GAME!")
        
    def replay():
        while True:
            play_again = input("Would you like to play again? [y/n]")
            if(play_again.lower() in ["yes", "y"]):
                return 1
            elif(play_again.lower() in ["no", "n"]):
                return 0
            else:
                print("Invalid input, only 'y'/'yes' or 'n'/'no' are valid")
                continue


# MAIN FUNCTION
def play():
    print("---------- WELCOME TO BLACKJACK 1.0 ----------")

    # Initialize main objects
    player1 = HumanPlayer(500)
    dealer = Dealer()
    wants_to_play = True
    
    while wants_to_play:
        # Create a new deck
        player1.reset_hand()
        dealer.reset_hand()
        my_deck = Deck()
    
        # For simplicity, I'm assuming that the dealer will always have money
        if player1.savings == 0:
            print("Player1, you don't have any more money!. GAME OVER.")
            break
            
        # Ask Player1 for their bet
        player1_bet = Utils.capture_bet()
        while not player1.can_bet(player1_bet):
            print(f"You only have ${player1.savings}. Go for a lower bet!")
            player1_bet = Utils.capture_bet()
        player1.bet(player1_bet)

        # Deal two cards to the Dealer and two cards to the Player
        player1.draw(my_deck)
        player1.draw(my_deck)   
        dealer.draw(my_deck)
        dealer.draw(my_deck)

        # Print current state of the board
        Utils.print_state(my_deck, dealer, player1, True)

        # Analyze if there're any aces already drawn by him, ask him what value will be use
        player1.define_aces_values()
        dealer.define_aces_values()

        # Reprint board in case anything changed
        Utils.print_state(my_deck, dealer, player1, True)
        
        # Ask the Player if they want to Hit or Stand
        while Utils.wants_to_hit():
            # He can draw an ACE here, consider that case
            player1.draw(my_deck)
            
            Utils.print_state(my_deck, dealer, player1, True)
            player1.define_aces_values()
            Utils.print_state(my_deck, dealer, player1, True)
            
            # Analyze if the player is busted
            if (player1.busted()):
                # The end of the game
                Utils.dealer_wins(my_deck, dealer, player1)
                break
        
        # If player1 is busted, we will ask him to play again
        if (player1.busted() and Utils.replay()):
            continue
                    
        # At this point player1 stood
        # If a Player Stands, play the Dealer's hand. The dealer will always 
        # Hit until the Dealer's value meets or exceeds 17
        while(dealer.hand_value() <= DEALERS_THRESHOLD):
            # hit
            dealer.draw(my_deck)
            Utils.print_state(my_deck, dealer, player1, False)

        if (dealer.busted()):
            Utils.player1_wins(my_deck, dealer, player1)
        else:
            # The one with more value has one
            if dealer.hand_value() > player1.hand_value():
                Utils.dealer_wins(my_deck, dealer, player1)
            elif dealer.hand_value() < player1.hand_value():
                Utils.player1_wins(my_deck, dealer, player1)
            else:
                # it's a tie, nobody wins or loses anything
                player1.reset_bet()
                Utils.print_state(my_deck, dealer, player1, False)
                print("")
                print("IT'S A TIED MATCH")

        if (Utils.replay()):
            continue
        else:
            break

# Main function
play()



    



    


Dealer
--------
Cards in deck: 47
Savings: $9223372036854775907
Ace of Clubs (11)|King of Hearts (10)|

Player1
--------
Savings: $400
Bet: $0
Seven of Hearts (7)|Two of Hearts (2)|Ten of Spades (10)|

PLAYER1 YOU'VE LOST THE GAME!
Would you like to play again? [y/n]n
